In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import export_graphviz
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df= pd.read_csv('../input/HR_comma_sep.csv')
df.head(10)

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
df['sales'].unique()

In [ ]:
df['salary'].unique()

In [ ]:
#Low=
#Medium=
#High=10
df['salary'].replace({'low':1,'medium':5,'high':10},inplace=True)

In [ ]:
satisfaction_level=df['satisfaction_level']
last_evaluation=df['last_evaluation']
number_project=df['number_project']
average_montly_hours=df['average_montly_hours']
time_spend_company=df['time_spend_company']
Work_accident=df['Work_accident']

In [ ]:
sns.pairplot(df, hue="left", vars=['satisfaction_level', 'last_evaluation', 'average_montly_hours','number_project'])
plt.show()

In [ ]:


#X_train,y_train,X_test,y_test=train_test_split(X,y,data_size=0.2)


corr=df.corr()
sns.heatmap(corr,annot=True)

In [ ]:
sns.pairplot(df, hue="salary", vars=['satisfaction_level','time_spend_company', 'last_evaluation', 'average_montly_hours','number_project'])
plt.show()

In [ ]:
#Creating dummy variables for the column:sales
dummies=pd.get_dummies(df['sales'],prefix='sales')
df=pd.concat([df,dummies],axis=1)
df.drop(['sales'],axis=1,inplace=True)
df.head(10)

Parameters:
=======
n_estimators=No of trees in the forest

n_jobs=The number of jobs to run in parallel for both fit and predict. If -1, then the number of jobs is set to the number of cores.

random_state=seed used by random number generator,This is for reproductibility

max_features=The number of features to consider when looking for the best split


 1. 'auto'=All features(n)
 2. 'sqrt'=sqrt(All features/n)
 3. 'log2'=lo2(n)
 4. None=n

min_sample_split=default=2

oob_score:TRUE/FALSE
Whether to use out-of-bag samples to estimate the generalization error.

In random forests there is no need of cross validation as this is internally done via oob(out of bag sampling).

In [ ]:
#Spilting data into test and train split:
X=df.drop(['left'],axis=1)
y=df['left']

In [ ]:
#RandomForestClassifier
model=RandomForestRegressor(n_estimators=100,n_jobs=-1,oob_score=True,random_state=19)

model.fit(X,y)

In [ ]:
#RandomForestClassifier
from sklearn.metrics import roc_auc_score


y_pred=model.oob_prediction_

acc=roc_auc_score(y,y_pred)
print('Accuracy :',acc)

In [ ]:
#K-Fold Validation
#Evaluating the model for a Ten-Fold Cross Validation

acc=[]
for i in [50,100,150,200,250,300,350,400,500,600]:
    #Training the model
    clf=RandomForestRegressor(n_estimators=i,n_jobs=-1,oob_score=True,random_state=19)
    clf.fit(X,y)
    y_pred=clf.oob_prediction_
    k=roc_auc_score(y,y_pred)
    acc.append(k)
   
print(acc)
    
    
    

In [ ]:
import matplotlib.pyplot as plt
xx=[50,100,150,200,250,300,350,400,500,600]
yy=acc
plt.plot(xx,yy)

In [ ]:
model.n_outputs_

In [ ]:
model.n_features_

In [ ]:
#Spilting data into test and train split:
X=df.drop(['left'],axis=1)
y=df['left']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [ ]:
print(X_train.shape,y_train.shape,)
print(X_test.shape,y_test.shape)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
#DecisionTreeClassifier
dt=DecisionTreeClassifier(max_depth=3,min_samples_leaf=int(0.05*len(X_train)),random_state=19)
boosted_dt=AdaBoostClassifier(dt,algorithm='SAMME',n_estimators=800,learning_rate=0.5)
boosted_dt.fit(X_train,y_train)
y_predicted=boosted_dt.predict(X_test)

print ("Area under ROC curve: %f"%(roc_auc_score(y_test, y_predicted)))
